# Test Client
## 1. Start Ollama cli and LLM 
To start serving ollama cli
``` 
ollama serve
``` 
To run pull and run a model - replace model name
``` 
ollama run MODEL_NAME
``` 

## 2. Set model variable
Change model_name string to name of model and version running on ollama

In [ ]:
model_name = "llama3.2:latest"

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

llm = Ollama(model=model_name, request_timeout=120.0)
Settings.llm = llm

In [ ]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

mcp_client = BasicMCPClient("http://mcp-service:8000/sse")
mcp_tools = McpToolSpec(client=mcp_client)

In [ ]:
tools = await mcp_tools.to_tool_list_async()
for tool in tools:
    print(tool.metadata.name, tool.metadata.description)

In [ ]:
from llama_index.core.agent.workflow import FunctionAgent

SYSTEM_PROMPT = "You are a helpful assistant that can use tools to answer questions."

async def get_agent():
    tool_list = await mcp_tools.to_tool_list_async()
    return FunctionAgent(
        name=f"{model_name}Agent",
        description=f"Agent using {model_name} and MCP tools",
        tools=tool_list,
        llm=llm,
        system_prompt=SYSTEM_PROMPT
    )

agent = await get_agent()

## Start agent
Run agent, inputting "Say hello" start message

In [ ]:
from llama_index.core.workflow import Context

ctx = Context(agent)

async def run_agent():
    message = "Say hello"
    response = await agent.run(message, ctx=ctx)
    print("Agent:", response)

await run_agent()

## Use Echo tool

In [ ]:
response = await agent.run("Please use the echo tool to repeat the message 'Testing 123'", ctx=ctx)
print("Agent:", response)
